In [1]:
from pyspark.sql import SparkSession


spark = SparkSession.builder \
    .appName("Read Inside Airbnb data") \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/02 12:29:35 WARN Utils: Your hostname, Dylans-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 10.20.42.250 instead (on interface en0)
25/08/02 12:29:35 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/02 12:29:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
listings = spark.read.csv("data/listings.csv.gz",
    header=True,
    inferSchema=True,
    sep=",",
    quote='"',
    escape='"',
    multiLine=True,
    mode="PERMISSIVE"
)

In [5]:
listings.show(20)

+------+--------------------+--------------+------------+---------------+--------------------+--------------------+---------------------+--------------------+-------+--------------------+------------+----------+--------------------+--------------------+------------------+------------------+--------------------+-----------------+--------------------+--------------------+--------------------+-------------------+-------------------------+--------------------+--------------------+----------------------+--------------------+----------------------+----------------------------+------------------+--------------------+--------------------+---------------+------------+---------+----------------+--------+----+--------------------+-------+--------------+--------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------------+----------------+----------------+---------------+---------------+---------------+-------------

In [8]:
for field in listings.schema:
    print(field)

StructField('id', LongType(), True)
StructField('listing_url', StringType(), True)
StructField('scrape_id', LongType(), True)
StructField('last_scraped', DateType(), True)
StructField('source', StringType(), True)
StructField('name', StringType(), True)
StructField('description', StringType(), True)
StructField('neighborhood_overview', StringType(), True)
StructField('picture_url', StringType(), True)
StructField('host_id', IntegerType(), True)
StructField('host_url', StringType(), True)
StructField('host_name', StringType(), True)
StructField('host_since', DateType(), True)
StructField('host_location', StringType(), True)
StructField('host_about', StringType(), True)
StructField('host_response_time', StringType(), True)
StructField('host_response_rate', StringType(), True)
StructField('host_acceptance_rate', StringType(), True)
StructField('host_is_superhost', StringType(), True)
StructField('host_thumbnail_url', StringType(), True)
StructField('host_picture_url', StringType(), True)


In [9]:
neighbourhoods = listings.select(listings.neighbourhood_cleansed)
neighbourhoods.show(20)

+----------------------+
|neighbourhood_cleansed|
+----------------------+
|             Islington|
|  Kensington and Ch...|
|           Westminster|
|            Wandsworth|
|  Richmond upon Thames|
|               Enfield|
|  Hammersmith and F...|
|                Camden|
|  Richmond upon Thames|
|               Lambeth|
|        City of London|
|                Camden|
|         Tower Hamlets|
|  Richmond upon Thames|
|              Haringey|
|  Hammersmith and F...|
|              Haringey|
|             Southwark|
|           Westminster|
|                Barnet|
+----------------------+
only showing top 20 rows


In [10]:
neighbourhoods = listings.select(listings.neighbourhood_cleansed.alias("neighbourhood"))
neighbourhoods.show(20, truncate=False)


+----------------------+
|neighbourhood         |
+----------------------+
|Islington             |
|Kensington and Chelsea|
|Westminster           |
|Wandsworth            |
|Richmond upon Thames  |
|Enfield               |
|Hammersmith and Fulham|
|Camden                |
|Richmond upon Thames  |
|Lambeth               |
|City of London        |
|Camden                |
|Tower Hamlets         |
|Richmond upon Thames  |
|Haringey              |
|Hammersmith and Fulham|
|Haringey              |
|Southwark             |
|Westminster           |
|Barnet                |
+----------------------+
only showing top 20 rows


In [11]:
review_locations = listings.select(listings.review_scores_location)
review_locations.show()

+----------------------+
|review_scores_location|
+----------------------+
|                  4.72|
|                  4.93|
|                  4.89|
|                  4.59|
|                  4.67|
|                  4.66|
|                  4.54|
|                  4.83|
|                  NULL|
|                  4.72|
|                  4.92|
|                  4.88|
|                  4.83|
|                  4.89|
|                  4.77|
|                  4.79|
|                   5.0|
|                  4.76|
|                   4.5|
|                  4.64|
+----------------------+
only showing top 20 rows


In [12]:
listings \
  .select(listings.review_scores_location) \
  .show()

+----------------------+
|review_scores_location|
+----------------------+
|                  4.72|
|                  4.93|
|                  4.89|
|                  4.59|
|                  4.67|
|                  4.66|
|                  4.54|
|                  4.83|
|                  NULL|
|                  4.72|
|                  4.92|
|                  4.88|
|                  4.83|
|                  4.89|
|                  4.77|
|                  4.79|
|                   5.0|
|                  4.76|
|                   4.5|
|                  4.64|
+----------------------+
only showing top 20 rows


In [13]:
high_score_listings = listings \
  .filter(listings.review_scores_location > 4.5) \
  .select('id', 'price', 'name', 'review_scores_location')

high_score_listings.show(20, truncate=False)

+------+-------+--------------------------------------------------+----------------------+
|id    |price  |name                                              |review_scores_location|
+------+-------+--------------------------------------------------+----------------------+
|13913 |$59.00 |Holiday London DB Room Let-on going               |4.72                  |
|15400 |$120.00|Bright Chelsea  Apartment. Chelsea!               |4.93                  |
|17402 |$493.00|Fab 3-Bed/2 Bath & Wifi: Trendy W1                |4.89                  |
|24328 |$190.00|Battersea live/work artist house, garden & parking|4.59                  |
|33332 |$140.00|Beautiful Ensuite Richmond-upon-Thames borough    |4.67                  |
|116268|$40.00 |Double Room (Unavailable for check in 31Dec-1Jan) |4.66                  |
|117203|$131.00|A stylish Victorian home in West London           |4.54                  |
|127652|$215.00|Contemporary central London apt                   |4.83                  |

In [14]:
high_score_listings.dropna().show(20, truncate=False)

+------+-------+--------------------------------------------------+----------------------+
|id    |price  |name                                              |review_scores_location|
+------+-------+--------------------------------------------------+----------------------+
|13913 |$59.00 |Holiday London DB Room Let-on going               |4.72                  |
|15400 |$120.00|Bright Chelsea  Apartment. Chelsea!               |4.93                  |
|17402 |$493.00|Fab 3-Bed/2 Bath & Wifi: Trendy W1                |4.89                  |
|24328 |$190.00|Battersea live/work artist house, garden & parking|4.59                  |
|33332 |$140.00|Beautiful Ensuite Richmond-upon-Thames borough    |4.67                  |
|116268|$40.00 |Double Room (Unavailable for check in 31Dec-1Jan) |4.66                  |
|117203|$131.00|A stylish Victorian home in West London           |4.54                  |
|127652|$215.00|Contemporary central London apt                   |4.83                  |

In [15]:
high_score_listings.schema['price']

StructField('price', StringType(), True)

In [16]:
from pyspark.sql.functions import regexp_replace

price_num_df = listings \
  .withColumn('price_num', regexp_replace('price', '[$,]', '').cast('float')) \

price_num_df.schema['price_num']

StructField('price_num', FloatType(), True)

In [43]:
price_num_df.select('price_num', 'name').where('price_num < 100').show(10, truncate=False)

+---------+--------------------------------------------------+
|price_num|name                                              |
+---------+--------------------------------------------------+
|59.0     |Holiday London DB Room Let-on going               |
|40.0     |Double Room (Unavailable for check in 31Dec-1Jan) |
|82.0     |You are GUARANTEED to love this                   |
|55.0     |Room 1 Large Double Bedroom - front ground floor  |
|55.0     |SPACIOUS ROOM IN CONTEMPORARY STYLE FLAT          |
|37.0     |West London-W7, Hanwell(Area Ealing) Room (Female)|
|69.0     |West London,loft ensuite, 5min2tube               |
|55.0     |A double Room 5mins from King's College Hospital  |
|94.0     |Charming Flat in Notting Hill                     |
|86.0     |Room with a view, shared flat,  central  Bankside |
+---------+--------------------------------------------------+
only showing top 10 rows


In [18]:
price_num_df.filter( (price_num_df.price_num < 100) & (price_num_df.review_scores_location > 4.5)) \
  .select('name', 'price', 'review_scores_location') \
  .show(truncate=False)

+--------------------------------------------------+------+----------------------+
|name                                              |price |review_scores_location|
+--------------------------------------------------+------+----------------------+
|Holiday London DB Room Let-on going               |$59.00|4.72                  |
|Double Room (Unavailable for check in 31Dec-1Jan) |$40.00|4.66                  |
|You are GUARANTEED to love this                   |$82.00|4.77                  |
|Room 1 Large Double Bedroom - front ground floor  |$55.00|5.0                   |
|SPACIOUS ROOM IN CONTEMPORARY STYLE FLAT          |$55.00|4.76                  |
|West London-W7, Hanwell(Area Ealing) Room (Female)|$37.00|4.64                  |
|West London,loft ensuite, 5min2tube               |$69.00|4.75                  |
|Charming Flat in Notting Hill                     |$94.00|4.87                  |
|Room with a view, shared flat,  central  Bankside |$86.00|4.86                  |
|You

In [19]:
price_num_df.filter('price_num < 100 AND review_scores_location > 4.5') \
  .select('name', 'price', 'review_scores_location') \
  .show(truncate=False)

+--------------------------------------------------+------+----------------------+
|name                                              |price |review_scores_location|
+--------------------------------------------------+------+----------------------+
|Holiday London DB Room Let-on going               |$59.00|4.72                  |
|Double Room (Unavailable for check in 31Dec-1Jan) |$40.00|4.66                  |
|You are GUARANTEED to love this                   |$82.00|4.77                  |
|Room 1 Large Double Bedroom - front ground floor  |$55.00|5.0                   |
|SPACIOUS ROOM IN CONTEMPORARY STYLE FLAT          |$55.00|4.76                  |
|West London-W7, Hanwell(Area Ealing) Room (Female)|$37.00|4.64                  |
|West London,loft ensuite, 5min2tube               |$69.00|4.75                  |
|Charming Flat in Notting Hill                     |$94.00|4.87                  |
|Room with a view, shared flat,  central  Bankside |$86.00|4.86                  |
|You

In [20]:
listings \
  .select(listings.property_type) \
  .distinct() \
  .show(truncate=False)

+----------------------------------+
|property_type                     |
+----------------------------------+
|Private room in lighthouse        |
|Private room in loft              |
|Private room in earthen home      |
|Entire chalet                     |
|Earthen home                      |
|Shared room in bus                |
|Farm stay                         |
|Entire rental unit                |
|Shared room in hostel             |
|Shared room                       |
|Private room in condo             |
|Room in boutique hotel            |
|Private room in religious building|
|Room in bed and breakfast         |
|Private room in casa particular   |
|Entire guesthouse                 |
|Private room in bungalow          |
|Entire cabin                      |
|Hut                               |
|Private room in nature lodge      |
+----------------------------------+
only showing top 20 rows


In [21]:
listings \
  .select(listings.property_type, listings.room_type) \
  .distinct() \
  .show(truncate=False)

+----------------------------------+---------------+
|property_type                     |room_type      |
+----------------------------------+---------------+
|Private room in casa particular   |Private room   |
|Room in hostel                    |Hotel room     |
|Dome                              |Entire home/apt|
|Entire serviced apartment         |Entire home/apt|
|Private room in loft              |Private room   |
|Shipping container                |Entire home/apt|
|Private room in villa             |Private room   |
|Farm stay                         |Entire home/apt|
|Room in hotel                     |Hotel room     |
|Shared room in rental unit        |Shared room    |
|Private room in guest suite       |Private room   |
|Room in rental unit               |Hotel room     |
|Room in serviced apartment        |Hotel room     |
|Private room in serviced apartment|Private room   |
|Private room in hostel            |Private room   |
|Shared room                       |Shared roo

In [22]:
listings \
  .select(listings.property_type) \
  .distinct() \
  .write \
  .csv('data/property_types')

In [24]:
listings.printSchema()

root
 |-- id: long (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: long (nullable = true)
 |-- last_scraped: date (nullable = true)
 |-- source: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: date (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_about: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_count: int

In [25]:
# 1. Get a non-null picture URL for any property ("picture_url" field)
# Select any non-null picture URL
listings.filter(
    listings.picture_url.isNotNull()
) \
.select('picture_url') \
.limit(1) \
.show(truncate=False)

+------------------------------------------------------------------------------------------------------+
|picture_url                                                                                           |
+------------------------------------------------------------------------------------------------------+
|https://a0.muscache.com/pictures/miso/Hosting-13913/original/d755aa6d-cebb-4464-80be-2722c921e8d5.jpeg|
+------------------------------------------------------------------------------------------------------+



In [26]:
# 2. Get number of properties that get more than 10 reviews per month
listings.filter(
  listings.reviews_per_month > 10
) \
.count()

66

In [27]:
listings.filter(
    (listings.bathrooms > listings.bedrooms)
) \
.select('name', 'bathrooms', 'bedrooms') \
.show(10, truncate=False)

+--------------------------------------------------+---------+--------+
|name                                              |bathrooms|bedrooms|
+--------------------------------------------------+---------+--------+
|Battersea live/work artist house, garden & parking|1.5      |1       |
|West London-W7, Hanwell(Area Ealing) Room (Female)|1.5      |1       |
|West London,loft ensuite, 5min2tube               |1.5      |1       |
|Large Bedroom with EnSuite Bathroom               |1.5      |1       |
|Cosy Double studio in Zone 2 Hammersmith (1)      |1.5      |1       |
|Shoreditch Loft                                   |1.5      |1       |
|Five minute walk to South Bank                    |1.5      |1       |
|Designer room Park Views 4 mins zone 1 station    |1.5      |1       |
|Cosy Double room London, River View               |1.5      |1       |
|Maisonette in Central London Zone 1               |1.5      |1       |
+--------------------------------------------------+---------+--

In [28]:
# 4. Get properties where the price is greater than 5,000. Collect the result as a Python list
from pyspark.sql.functions import regexp_replace

listings_with_price = listings \
  .withColumn('price_numeric', regexp_replace('price', '[$,]', '').cast('float'))

res = listings_with_price.filter(
    (listings_with_price.price_numeric > 5000)
) \
.select('name', 'price') \
.collect()

res

[Row(name='Bright ,Modern, 12m to Bond street.', price='$5,500.00'),
 Row(name='Room in a cosy flat. Central, clean', price='$8,000.00'),
 Row(name='3 Bed Flat in South Hampstead with Large Garden!', price='$25,000.00'),
 Row(name='Spacious Private Ground Floor Room', price='$7,693.00'),
 Row(name='No Longer Available', price='$53,588.00'),
 Row(name='Very nice double room in the heart of Soho', price='$5,100.00'),
 Row(name='Knightsbridge Penthouse', price='$8,895.00'),
 Row(name='Luxury 5-star Flat with Art & Tech', price='$6,276.00'),
 Row(name='The Apartments by The Sloane Club, L 2 Bedroom Apt', price='$7,589.00'),
 Row(name='The Apartments by The Sloane Club, One Bedroom Apt', price='$7,589.00'),
 Row(name='Great Apartment next to Sloane Square', price='$6,250.00'),
 Row(name='Lovely Private double room at zone 2 Holloway Road', price='$5,147.00'),
 Row(name='Kensington- Luxury 2 bedroom ground floor flat', price='$8,000.00'),
 Row(name='Single room. 7ft x 9ft - Over looking gard

In [29]:
# 5. Get a list of properties with the following characteristics:
# * price < 150
# * more than 20 reviews
# * review_scores_rating > 4.5
# Consider using the "&" operator

listings_with_price.filter(
    (listings_with_price.price_numeric < 150) &
    (listings_with_price.number_of_reviews > 20) &
    (listings_with_price.review_scores_rating > 4.5)
  ) \
.select('name', 'price_numeric', 'number_of_reviews', 'review_scores_rating') \
.show(truncate=False)

+--------------------------------------------------+-------------+-----------------+--------------------+
|name                                              |price_numeric|number_of_reviews|review_scores_rating|
+--------------------------------------------------+-------------+-----------------+--------------------+
|Holiday London DB Room Let-on going               |59.0         |44               |4.82                |
|Bright Chelsea  Apartment. Chelsea!               |120.0        |96               |4.8                 |
|Double Room (Unavailable for check in 31Dec-1Jan) |40.0         |38               |4.89                |
|A stylish Victorian home in West London           |131.0        |91               |4.85                |
|I Bedroom flat Tower of London                    |145.0        |248              |4.9                 |
|You are GUARANTEED to love this                   |82.0         |691              |4.86                |
|SPACIOUS ROOM IN CONTEMPORARY STYLE FLAT     

In [30]:
# 7. Get the highest listing price in this dataset
# Consider using the "max" function from "pyspark.sql.functions"

from pyspark.sql.functions import max


listings_with_price \
  .select(max('price_numeric')) \
  .show()

+------------------+
|max(price_numeric)|
+------------------+
|           80000.0|
+------------------+



In [31]:
res = listings_with_price \
  .select(max('price_numeric').alias('max_price')) \
  .collect()


max_price = res[0]['max_price']
max_price

80000.0

In [32]:
from pyspark.sql.functions import year

listings.filter(
    year(listings.first_review) == 2024
) \
.select('name', 'first_review') \
.show(10, truncate=False)

+--------------------------------------------------+------------+
|name                                              |first_review|
+--------------------------------------------------+------------+
|Close to Wimbledon All England Tennis -huge double|2024-08-11  |
|one Double bed room with en-suite facilities      |2024-03-21  |
|Double Room for one -Herne Hill Zone 2/3Thameslink|2024-05-26  |
|Apartment in Southwark - Private living room      |2024-07-09  |
|Sm double room  with own bathroom                 |2024-06-04  |
|Superlux flat in Knightsbridge                    |2024-01-01  |
|Luxurious Flat in South Kensington                |2024-06-19  |
|The Pink House, Notting Hill                      |2024-07-14  |
|Superior Single Room (En-suite)                   |2024-06-21  |
|Charming flat in Tufnell Park                     |2024-04-28  |
+--------------------------------------------------+------------+
only showing top 10 rows


In [33]:
listings.printSchema()

root
 |-- id: long (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: long (nullable = true)
 |-- last_scraped: date (nullable = true)
 |-- source: string (nullable = true)
 |-- name: string (nullable = true)
 |-- description: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: date (nullable = true)
 |-- host_location: string (nullable = true)
 |-- host_about: string (nullable = true)
 |-- host_response_time: string (nullable = true)
 |-- host_response_rate: string (nullable = true)
 |-- host_acceptance_rate: string (nullable = true)
 |-- host_is_superhost: string (nullable = true)
 |-- host_thumbnail_url: string (nullable = true)
 |-- host_picture_url: string (nullable = true)
 |-- host_neighbourhood: string (nullable = true)
 |-- host_listings_count: int

In [34]:
listings.select('first_review').show(10, truncate=False)

+------------+
|first_review|
+------------+
|2010-08-18  |
|2009-12-21  |
|2011-03-21  |
|2010-11-15  |
|2010-10-16  |
|2013-07-06  |
|2012-01-04  |
|2011-07-03  |
|NULL        |
|2011-08-29  |
+------------+
only showing top 10 rows


In [35]:
listings \
  .groupby(listings.property_type) \
  .count() \
  .show(truncate=False)

+----------------------------------+-----+
|property_type                     |count|
+----------------------------------+-----+
|Private room in lighthouse        |2    |
|Private room in loft              |171  |
|Private room in earthen home      |3    |
|Entire chalet                     |4    |
|Earthen home                      |2    |
|Shared room in bus                |1    |
|Farm stay                         |4    |
|Entire rental unit                |39372|
|Shared room in hostel             |59   |
|Shared room                       |4    |
|Private room in condo             |3515 |
|Room in boutique hotel            |239  |
|Private room in religious building|4    |
|Room in bed and breakfast         |24   |
|Private room in casa particular   |86   |
|Entire guesthouse                 |228  |
|Private room in bungalow          |64   |
|Entire cabin                      |39   |
|Hut                               |2    |
|Private room in nature lodge      |2    |
+----------

In [36]:
import pyspark.sql.functions as F

listings \
  .groupby(listings.property_type) \
  .agg(
    F.count('property_type').alias('count')
  ) \
  .orderBy('count', ascending=[False]) \
  .show(truncate=False)

+----------------------------------+-----+
|property_type                     |count|
+----------------------------------+-----+
|Entire rental unit                |39372|
|Private room in rental unit       |14837|
|Private room in home              |11835|
|Entire condo                      |8800 |
|Entire home                       |8756 |
|Private room in condo             |3515 |
|Entire serviced apartment         |1852 |
|Private room in townhouse         |1294 |
|Entire townhouse                  |1123 |
|Room in hotel                     |783  |
|Private room in bed and breakfast |486  |
|Entire loft                       |367  |
|Private room in guesthouse        |316  |
|Room in boutique hotel            |239  |
|Entire guesthouse                 |228  |
|Shared room in rental unit        |191  |
|Entire guest suite                |181  |
|Private room in guest suite       |178  |
|Private room in loft              |171  |
|Private room in serviced apartment|144  |
+----------

In [37]:
listings \
  .groupby(listings.property_type) \
  .agg(
    F.count('property_type').alias('count'),
    F.avg('review_scores_location')
  ) \
  .orderBy('count', ascending=[False]) \
  .show(truncate=False)

+----------------------------------+-----+---------------------------+
|property_type                     |count|avg(review_scores_location)|
+----------------------------------+-----+---------------------------+
|Entire rental unit                |39372|4.72794867465818           |
|Private room in rental unit       |14837|4.732797964376552          |
|Private room in home              |11835|4.6990531037827195         |
|Entire condo                      |8800 |4.763906656266228          |
|Entire home                       |8756 |4.722269211782102          |
|Private room in condo             |3515 |4.7656638325703            |
|Entire serviced apartment         |1852 |4.7218975332068345         |
|Private room in townhouse         |1294 |4.75918656056587           |
|Entire townhouse                  |1123 |4.812793814432991          |
|Room in hotel                     |783  |4.649164619164618          |
|Private room in bed and breakfast |486  |4.7173976608187145         |
|Entir

In [38]:
reviews = spark.read.csv("data/reviews.csv.gz",
    header=True,
    inferSchema=True,
    sep=",",
    quote='"',
    escape='"',
    multiLine=True,
    mode="PERMISSIVE"
)

In [39]:
listings_reviews = listings.join(
    reviews, listings.id == reviews.listing_id, how='inner'
)

In [42]:
reviews_per_listing = listings_reviews \
  .groupBy(listings.id, listings.name) \
  .agg(
    F.count(reviews.id).alias('num_reviews')
  ) \
  .orderBy('num_reviews', ascending=False) \
  .show(truncate=False)


+--------+--------------------------------------------------+-----------+
|id      |name                                              |num_reviews|
+--------+--------------------------------------------------+-----------+
|30760930|Double Garden View room - London House Hotel***   |1682       |
|47408549|Double Room+ Ensuite                              |1650       |
|43120947|Private double room with en suite facilities      |1553       |
|19670926|Designer Studio Apartment in Central London       |1382       |
|1436172 |Cosy Double in Kings Cross Houseshare nr Eurostar |1134       |
|45006692|Budget Double Room In Colliers Hotel.             |1132       |
|2126708 |London's best transport hub 5 mins walk! Safe too!|1071       |
|1436177 |En-suite Double in Kings Cross Houseshare Eurostar|943        |
|3855375 |Double in Kings Cross Houseshare nr Eurostar      |935        |
|2659707 |Large Room + Private Bathroom, E3.                |893        |
|42081759|Micro Studio at Locke at Bro